In [158]:
import timeit

In [1]:
def permutations(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            rest = lst[:i] + lst[i+1:]
            for p in permutations(rest):
                yield [lst[i]] + p

In [2]:
def _permutations_multi(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_multi(rest):
                    yield [lst[i]] + p

In [3]:
def permutations_multi(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    yield from _permutations_multi(lst)

In [5]:
from server.board import *
from server.board_type import *

In [14]:
def create_boards(board_type, filename):
    output_file = open(filename, "w")
    
    for board_objects in permutations_multi(board_type.num_objects):
        board = Board(board_objects)
        if board.check_constraints(board_type.constraints):
            output_file.write(str(board) + "\n")
            
    output_file.close()

In [16]:
list(permutations_multi({"A": 4, "B": 3, None: 2}))

[['A', 'A', 'A', 'A', 'B', 'B', 'B', None, None],
 ['A', 'A', 'A', 'A', 'B', 'B', None, 'B', None],
 ['A', 'A', 'A', 'A', 'B', 'B', None, None, 'B'],
 ['A', 'A', 'A', 'A', 'B', None, 'B', 'B', None],
 ['A', 'A', 'A', 'A', 'B', None, 'B', None, 'B'],
 ['A', 'A', 'A', 'A', 'B', None, None, 'B', 'B'],
 ['A', 'A', 'A', 'A', None, 'B', 'B', 'B', None],
 ['A', 'A', 'A', 'A', None, 'B', 'B', None, 'B'],
 ['A', 'A', 'A', 'A', None, 'B', None, 'B', 'B'],
 ['A', 'A', 'A', 'A', None, None, 'B', 'B', 'B'],
 ['A', 'A', 'A', 'B', 'A', 'B', 'B', None, None],
 ['A', 'A', 'A', 'B', 'A', 'B', None, 'B', None],
 ['A', 'A', 'A', 'B', 'A', 'B', None, None, 'B'],
 ['A', 'A', 'A', 'B', 'A', None, 'B', 'B', None],
 ['A', 'A', 'A', 'B', 'A', None, 'B', None, 'B'],
 ['A', 'A', 'A', 'B', 'A', None, None, 'B', 'B'],
 ['A', 'A', 'A', 'B', 'B', 'A', 'B', None, None],
 ['A', 'A', 'A', 'B', 'B', 'A', None, 'B', None],
 ['A', 'A', 'A', 'B', 'B', 'A', None, None, 'B'],
 ['A', 'A', 'A', 'B', 'B', 'B', 'A', None, None],


In [18]:
len({"A": 4, "B": 3, None: 2}.keys())

3

In [44]:
def _permutations_no_touch(lst):
    if len(lst) == 0:
        return
    elif len(lst) == 1:
        yield lst
    else:
        for i in range(len(lst)):
            if i == 0 or lst[i] != lst[i-1]:
                rest = lst[:i] + lst[i+1:]
                for p in _permutations_no_touch(rest):
                    if p[0] == lst[i] or p[0] is None or lst[i] is None:
                        yield [lst[i]] + p

def permutations_no_touch(counts, empty):
    if len(counts.keys()) > empty:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
        
        for p in _permutations_no_touch(lst):
            if p[0] == p[-1] or p[0] is None or p[-1] is None:
                yield p

In [153]:
def sum_counts(d):
    s = 0
    for k in d:
        s += d[k]
    return s
    
def _fill_no_touch(lst, is_hole):
    if len(lst) == 0:
        if all(is_hole):
            yield [None]*len(is_hole)
        return
    else:
        if is_hole[0]:
            for p in _fill_no_touch(lst, is_hole[1:]):
                yield [None] + p
        else:
            for i in range(len(lst)):
                if i == 0 or lst[i] != lst[i-1]:
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_touch(rest, is_hole[1:]):
                        if len(p) == 0 or p[0] == lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_touch(counts, board):
    is_hole = [obj is not None and board[i-1] is None for i, obj in enumerate(board)]
    is_hole_collapsed = [hole_val for i, hole_val in enumerate(is_hole) if hole_val or board[i] is None]
    holes = sum(is_hole)
    empty = len([obj for obj in board if obj is None]) - sum_counts(counts)
    gaps = holes + empty
    
    if len(counts.keys()) > gaps:
        return []
    else:
        lst = []
        for val in counts:
            for i in range(counts[val]):
                lst.append(val)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_touch(lst, is_hole_collapsed):
            if p[0] == p[-1] or p[0] is None or p[-1] is None or p[0] not in counts or p[-1] not in counts:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [289]:
def fill_no_touch2(counts, board):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    perms = permutations_multi({**counts, None: num_none})
    boards = []
    for p in perms:
        board_copy = board.copy()
        j = 0
        for i in range(len(board)):
            if board[i] is None:
                board_copy[i] = p[j]
                j += 1
        
        if all([board_copy[i] == board_copy[i-1] or board_copy[i] not in counts or board_copy[i-1] not in counts
               for i in range(len(board_copy))]):
            boards.append(board_copy)
    return boards

In [546]:
def fill_every_touch(obj1, obj2, counts, board):
    

In [547]:
list(fill_every_touch("A", "B", {"A": 2, "B": 3}, [None]*5))

[]

In [552]:
list(fill_no_touch({"A": 3, "B": 3}, [None, None, "B", "D", None, "B", None, None, "E", "F", None]))

[]

In [536]:
fill_no_touch2({"A": 2, "B": 3}, [None, None, "B", "D", None, None, None, "E", "F", None])

[['A', None, 'B', 'D', 'B', 'B', 'B', 'E', 'F', 'A'],
 ['B', 'B', 'B', 'D', 'A', 'A', None, 'E', 'F', 'B'],
 ['B', 'B', 'B', 'D', 'A', None, 'A', 'E', 'F', 'B'],
 ['B', 'B', 'B', 'D', None, 'A', 'A', 'E', 'F', 'B']]

In [537]:
def multi_alternate(counts):
    lst = []
    for val in counts:
        for i in range(counts[val]):
            lst.append(val)
    tpl = tuple(lst)
    perms = itertools.permutations(tpl)
    return set(perms)

In [538]:
timeit.timeit('fill_no_touch2({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=1000, globals=globals())

0.3251120299973991

In [544]:
timeit.timeit('fill_no_touch({"A": 2, "B": 3}, [None, None, "A", "D", None, None, None, "E", "F", None])', number=1000000, globals=globals())

0.291645797995443

In [190]:
timeit.timeit("permutations_multi({'A': 3, 'B': 4})", number=100000, globals=globals())

0.03242176799903973

In [167]:
timeit.timeit("multi_alternate({'A': 3, 'B': 4})", number=100000, globals=globals())

26.60144549100005

In [719]:
def _fill_no_within(prev, countdown, counts, no_within_objs, board, n, i):
    if (i == len(board)):
        yield []
        return
    
    if board[i] != None:
        if board[i] in no_within_objs:
            if board[i] != prev and countdown != 0:
                return
            new_prev = board[i]
            new_countdown = n
        else:
            new_prev = prev
            new_countdown = countdown - 1
            
        for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
            yield [board[i]] + p
        return
        
    obj_choices = list(counts.keys())
    for obj in obj_choices:
        restricted_obj = obj in no_within_objs
        if obj == prev or not restricted_obj or countdown == 0:
            counts[obj] -= 1
            if counts[obj] == 0:
                del counts[obj]
                
            if restricted_obj:
                new_countdown = n
                new_prev = obj
            else:
                new_countdown = max(0, countdown - 1)
                new_prev = prev
                
            for p in _fill_no_within(new_prev, new_countdown, counts, no_within_objs, board, n, i+1):
                yield [obj] + p
            
            if obj in counts:
                counts[obj] += 1
            else:
                counts[obj] = 1

def fill_no_within(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in _fill_no_within(None, 0, {**counts, None: num_none}, counts.keys(), board, n, 0):
        first_i = next(i for i, obj in enumerate(p) if obj in counts)
        last_i = next(i for i in range(len(p) - 1, -1, -1) if p[i] in counts and p[i] != p[first_i])
        diff = first_i + (len(p) - last_i)
        if diff > n:
            yield p

In [720]:
def fill_no_within2(counts, board, n):
    num_none = len([obj for obj in board if obj is None]) - sum_counts(counts)
    for p in permutations_multi({**counts, None: num_none}):
        j = 0 
        board_copy = board.copy()
        for i in range(len(board)):
            if board[i] is None:
                board_copy[i] = p[j]
                j += 1
        
        prev = None
        countdown = 0
        is_valid = True
        for i in range(-n, len(board_copy)):
            obj = board_copy[i]
            if obj in counts:
                if countdown != 0 and obj != prev:
                    is_valid = False
                    break
                prev = obj
                countdown = n
            else:
                countdown = max(0, countdown - 1)
        
        if is_valid:
            yield board_copy

In [721]:
b = ["A", None, None, None, None, None, None, None]

In [722]:
list(fill_no_within({"A": 1, "B": 1}, b, 2))

[['A', 'A', None, None, 'B', None, None, None],
 ['A', 'A', None, None, None, 'B', None, None],
 ['A', None, None, None, 'B', None, None, 'A'],
 ['A', None, None, 'B', None, None, None, 'A'],
 ['A', None, None, 'B', None, None, 'A', None],
 ['A', None, 'A', None, None, 'B', None, None]]

In [723]:
list(fill_no_within2({"A": 1, "B": 1}, b, 2))

[['A', 'A', None, None, 'B', None, None, None],
 ['A', 'A', None, None, None, 'B', None, None],
 ['A', None, 'A', None, None, 'B', None, None],
 ['A', None, None, 'B', None, None, 'A', None],
 ['A', None, None, 'B', None, None, None, 'A'],
 ['A', None, None, None, 'B', None, None, 'A']]

In [724]:
timeit.timeit('list(fill_no_within2({"A": 1, "B": 1}, b, 2))', number=1000, globals=globals())

0.2317381509928964

In [729]:
timeit.timeit('list(fill_no_within({"A": 1, "B": 1}, b, 2))', number=10000, globals=globals())

0.8494315559946699

In [ ]:
def _fill_within(obj1, obj2, board, countdown, n, i):
    if i == len(board):
        yield []
        
    if board[i] is not None:
        if countdown == 0:
            

def fill_within(obj1, obj2, board):
    

In [758]:
def _fill_no_self_touch(lst, is_hole):
    if len(lst) == 0:
        if all(is_hole):
            yield [None]*len(is_hole)
        return
    else:
        if is_hole[0]:
            for p in _fill_no_self_touch(lst, is_hole[1:]):
                yield [None] + p
        else:
            for i in range(len(lst)):
                if i == 0 or lst[i] != lst[i-1]:
                    rest = lst[:i] + lst[i+1:]
                    for p in _fill_no_self_touch(rest, is_hole[1:]):
                        if len(p) == 0 or p[0] != lst[i] or p[0] is None or lst[i] is None:
                            yield [lst[i]] + p

def fill_no_self_touch(obj, num_obj, board):
    is_hole = [o is not None and board[i-1] is None for i, o in enumerate(board)]
    is_hole_collapsed = [hole_val for i, hole_val in enumerate(is_hole) if hole_val or board[i] is None]
    holes = sum(is_hole)
    empty = len([o for o in board if o is None]) - num_obj
    gaps = holes + empty
    
    if num_obj > gaps:
        return []
    else:
        lst = []
        for i in range(num_obj):
            lst.append(obj)
        for i in range(empty):
            lst.append(None)
                    
        for p in _fill_no_self_touch(lst, is_hole_collapsed):
            if p[0] != p[-1] or p[0] is None or p[-1] is None or p[0] != obj or p[-1] != obj:
                board_copy = board.copy()
                j = 0
                for i in range(len(board_copy)):
                    if board[i] is None:
                        board_copy[i] = p[j]
                        j += 1
                    elif board[i-1] is None:
                        j += 1
                yield board_copy

In [770]:
def _fill_no_self_touch(prev, holes, obj, num_obj, num_none, t=0):
    if num_obj == 0 and num_none == 0:
        if all(holes):
            if len(holes) == 0 or \
            (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
                yield [None]*len(holes)
        return
    else:
        if holes[0] and (holes[0][0] != prev or holes[0][0] != obj or prev != obj):
            for p in _fill_no_self_touch(holes[0][-1], holes[1:], obj, num_obj, num_none, t+1):
                yield [None] + p
        else:
            if num_obj > 0 and obj != prev:
                for p in _fill_no_self_touch(obj, holes[1:], obj, num_obj - 1, num_none, t+1):
                    yield [obj] + p
            if num_none > 0:
                for p in _fill_no_self_touch(None, holes[1:], obj, num_obj, num_none - 1, t+1):
                    yield [None] + p

def fill_no_self_touch(obj, num_obj, board):
    holes = []
    in_hole = False
    for o in board:
        if o is None:
            holes.append(False)
            in_hole = False
        elif in_hole:
            holes[-1] += (o,)
        else:
            in_hole = True
            holes.append((o,))
                        
    num_holes = len([val for val in holes if val != False])
    empty = len([o for o in board if o is None]) - num_obj
    gaps = num_holes + empty
    
    if num_obj > gaps:
        return []
    else:                     
        for p in _fill_no_self_touch(None, holes, obj, num_obj, empty):
            if p[0] != p[-1] or p[0] != obj or p[-1] != obj:
                if holes[-1] == False or holes[0] != False or p[0] != holes[-1][-1] \
                or p[0] != obj or holes[-1][-1] != obj:
                    board_copy = board.copy()
                    j = 0
                    for i in range(len(board_copy)):
                        if board[i] is None:
                            board_copy[i] = p[j]
                            j += 1
                        elif board[i-1] is None:
                            j += 1
                    yield board_copy

In [782]:
b = Board([None, None, None, None, None, "A", "B"])

In [783]:
list(fill_no_self_touch("A", 2, b))

[<Board A-A--AB>, <Board A--A-AB>, <Board -A-A-AB>]

In [743]:
timeit.timeit("list(fill_no_self_touch('A', 2, b))", number=1000, globals=globals())

0.17030155699467286